###Setup & Statistical Profiling
**Task**: Calculate statistical summaries. **Concept**: .describe() gives us the "Five Number Summary". It is the fastest way to catch data quality issues (like negative prices).

In [0]:
from pyspark.sql.functions import col, dayofweek, hour, log, unix_timestamp, first, when, count, avg, lit, corr
from pyspark.sql.window import Window

# Setup context
spark.sql("USE CATALOG course_catalog")
spark.sql("USE SCHEMA ecommerce_governed")

# Load Silver Data (Cleaned events)
df_events = spark.table("silver_events")

print("📊 STATISTICAL PROFILE (Price):")
# 1. Descriptive Statistics for Price
# We filter out free items to get a true sense of 'products for sale'
df_events.filter(col("price") > 0).select("price").summary().show()

print("🔍 outlier Detection:")
# Find the most expensive items to verify they are real
display(
    df_events.orderBy(col("price").desc()).limit(5).select("product_id", "category_code", "brand", "price")
)

###Hypothesis Testing (Weekday vs. Weekend)
**Task**: Test if conversion rates differ between weekdays and weekends. **Concept**: We create a flag is_weekend (Sunday=1, Saturday=7) and compare the Purchase/View ratio.

In [0]:
# ---------------------------------------------------------
# HYPOTHESIS TEST: "Weekend Warriors"
# Assumption: People browse on weekdays but buy on weekends.
# ---------------------------------------------------------

# 1. Feature Creation: Add Weekend Flag
# Spark dayofweek: 1=Sunday, 7=Saturday
df_hypothesis = df_events.withColumn(
    "is_weekend", 
    when(dayofweek(col("event_date")).isin([1, 7]), "Weekend")
    .otherwise("Weekday")
)

# 2. Aggregation: Calculate Conversion Rate per Group
df_test_results = df_hypothesis.groupBy("is_weekend").agg(
    count(when(col("event_type") == "view", 1)).alias("views"),
    count(when(col("event_type") == "purchase", 1)).alias("purchases")
).withColumn(
    "conversion_rate_pct", 
    (col("purchases") / col("views")) * 100
)

print("🧪 HYPOTHESIS RESULTS:")
display(df_test_results)

# VISUALIZATION TIP:
# Create a Bar Chart. X-Axis: is_weekend. Y-Axis: conversion_rate_pct.
# This visually proves/disproves if weekends are better for sales.

Databricks visualization. Run in Databricks to view.

###Correlation Analysis
**Task**: Identify correlations. **Concept**: corr() calculates the Pearson Correlation Coefficient (-1 to 1). A value near -1 means "As Price goes UP, Sales go DOWN."

In [0]:
# ---------------------------------------------------------
# CORRELATION CHECK
# Question: Does price negatively impact the volume of purchases?
# ---------------------------------------------------------

# We need to aggregate first because 'purchase' is an event, 
# not a number on the view row.
product_stats = df_events.groupBy("product_id").agg(
    avg("price").alias("avg_price"),
    count(when(col("event_type") == "purchase", 1)).alias("purchase_count")
)

# Calculate Correlation
correlation_val = product_stats.stat.corr("avg_price", "purchase_count")

print(f"📉 Price vs. Sales Volume Correlation: {correlation_val:.4f}")
print("""   (Interpretation: Values close to 0 mean weak relationship,
       close to -1 mean strong inverse relationship)""")

# Visual Scatter Plot
display(product_stats.sample(0.01).select("avg_price", "purchase_count"))
# Plot Tip: Scatter Plot. X=avg_price, Y=purchase_count.

Databricks visualization. Run in Databricks to view.

###Feature Engineering for ML
**Task**: Engineer features (Log Price, Time Lags). **Concept**:

* **Log Price**: Prices are "right-skewed" (many cheap items, few super expensive ones). Taking the log() makes the distribution normal, which helps ML models learn better.

* **Time Since First View**: This is a powerful "Behavioral Feature." A user who has been browsing for 10 minutes is more likely to buy than someone who just arrived.

In [0]:
# ---------------------------------------------------------
# FEATURE ENGINEERING PIPELINE
# Goal: Create a rich dataset for tomorrow's ML Model
# ---------------------------------------------------------

# 1. Define Window for Behavioral Features
# Partition by User, Ordered by Time
user_window = Window.partitionBy("user_id").orderBy("event_time")

# 2. Create Features
df_features = df_events \
    .withColumn("hour_of_day", hour("event_time")) \
    .withColumn("day_of_week", dayofweek("event_date")) \
    .withColumn("is_weekend_flag", when(col("day_of_week").isin([1, 7]), 1).otherwise(0)) \
    .withColumn("price_log", log(col("price") + 1)) \
    .withColumn("time_since_first_interaction", 
                # Current Time - First Time seen in this window
                unix_timestamp("event_time") - 
                unix_timestamp(first("event_time").over(user_window))
    )

# 3. Save Feature Store (Ready for ML)
table_name = "gold_ml_features"
print(f"💾 Saving Engineered Features to: {table_name}...")

df_features.select(
    "user_id", "product_id", "brand", "category_code",
    "hour_of_day", "day_of_week", "is_weekend_flag", "price_log", "time_since_first_interaction",
    "event_type" # Target Variable
).write.mode("overwrite").saveAsTable(table_name)

print("✅ Feature Engineering Complete.")
display(spark.table(table_name).limit(5))